## 1. Introduction of PP-ShiTu
PP-ShiTu is a practical lightweight general-purpose image recognition system, which mainly consists of three modules: subject detection, feature learning and vector retrieval. The system adopts a variety of strategies from 8 aspects of backbone network selection and adjustment, loss function selection, data enhancement, learning rate transformation strategy, regularization parameter selection, use of pre-trained models, and model pruning and quantification Optimization, and finally a system that can complete the image recognition of the 10w+ library in only 0.2s on the CPU.
For more details, please refer to [PP-ShiTu Technical Paper](https://arxiv.org/pdf/2111.00775.pdf).

Learn more about PaddleClas at https://github.com/PaddlePaddle/PaddleClas.

## 2. Preview and application scenarios
### 2.1 product recognition：

#### 2.1.1 dataset：

The training data set and test set of PP-ShiTu are composed of 7 data sets such as Aliproduct and GLDv2. For details, please refer to [PP-ShiTu Experimental Part](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.4/docs/zh_CN/image_recognition_pipeline/feature_extraction.md#4-%E5%AE%9E%E9%AA%8C%E9%83%A8%E5%88%86)

#### 2.1.2 output preview：

The detection effect of PP-ShiTu on the picture is as follows

![](https://github.com/PaddlePaddle/PaddleClas/raw/release/2.4/docs/images/recognition/drink_data_demo/output/nongfu_spring.jpeg)

## 3. How to use

### 3.1 model inference：

- download PaddleClas

In [ ]:
# When not running on the Jupyter Notebook, you need to add the "!" and "%" statement comments, do not need to run.
%cd ~/work

# Clone PaddleClas（Cloning is faster on gitee）
!git clone https://gitee.com/paddlepaddle/PaddleClas

- Install PaddleClas and its dependencies

In [ ]:
# Access into PaddleClas
%cd ~/work/PaddleClas/

# Switch to the 2.4 branch
!git checkout release/2.4

# Install required dependencies
!pip install -r requirements.txt

# set GPU environment
# %env CUDA_VISIBLE_DEVICES=0

- Quick start

Congratulations! You have successfully installed PaddleClas, now you can experience the image recognition as guided below.

In [ ]:
# Access into PaddleClas
%cd ~/work/PaddleClas/

# Create a folder for storing mainbody detection and feature extraction inference models
%mkdir -p deploy/models

# Access into models
%cd deploy/models

# Download the mainbody detection inference model and unzip it
!wget -nc https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/rec/models/inference/picodet_PPLCNet_x2_5_mainbody_lite_v1.0_infer.tar && tar -xf picodet_PPLCNet_x2_5_mainbody_lite_v1.0_infer.tar

# Download the feature extraction inference model and unzip it
!wget -nc https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/rec/models/inference/general_PPLCNet_x2_5_lite_v1.0_infer.tar && tar -xf general_PPLCNet_x2_5_lite_v1.0_infer.tar

# Back to deloy
%cd ~/work/PaddleClas/deploy/

# Download the test data drink_dataset_v1.0 and unzip it
!wget -nc https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/rec/data/drink_dataset_v1.0.tar && tar -xf drink_dataset_v1.0.tar

In [ ]:
# Access into PaddleClas
%cd ~/work/PaddleClas/

# Access into deploy
%cd ./deploy

# general_PPLCNet_x2_5_lite_v1.0 inference model is used to extract the features of gallery images and make a retrieval database
!python3.7 python/build_gallery.py -c configs/inference_general.yaml -o Global.rec_inference_model_dir=./models/general_PPLCNet_x2_5_lite_v1.0_infer

# Recognition inference for nongfu_spring.jpeg image (GPU inference)
!python3.7 python/predict_system.py -c configs/inference_general.yaml
# Recognition inference for nongfu_spring.jpeg image (CPU inference)
!python3.7 python/predict_system.py -c configs/inference_general.yaml -o Global.use_gpu=False

At the same time, the recognition results (with detection bounding box, predicted class name and similarity) will be saved to `PaddleClas/deploy/output/nongfu_spring.jpeg`, as [2.1.2 output preview](#212-output-preview) displayed.

![](https://github.com/PaddlePaddle/PaddleClas/raw/release/2.4/docs/images/recognition/drink_data_demo/output/nongfu_spring.jpeg)

### 3.2 Model training

- clone PaddleClas repo(refer to [3.1 model inference](#31-model-inference)), and checkout to release/2.4 branch
- For the dataset preparation, training, evaluation and other steps of the mainbody detection model, please refer to  [PP-ShiTu mainbody detection doc](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.4/docs/zh_CN/image_recognition_pipeline/mainbody_detection.md)
- For the dataset preparation, training, evaluation and other steps of the mainbody detection model, please refer to  [PP-ShiTu feature extraction doc](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.4/docs/zh_CN/image_recognition_pipeline/feature_extraction.md#51-%E6%95%B0%E6%8D%AE%E5%87%86%E5%A4%87)


## 4. Algorithm
PP-ShiTu series recognition systems, including PP-ShiTu introduced in this document, consists of three modules to complete the entire recognition process, as shown in the figure below

![PP-ShiTu System](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.4/docs/images/structure.jpg?raw=true)

- Mainbody detection: The blue colored module in the figure above detects potential targets in the input image, and then cropping these targets, filtering unimportant backgrounds, and reducing background interference. In fact, this practice of retaining the mainbody and filtering the background is a simple, effective and widely used method in practice.
- Feature extraction: Receive the cropped image containing the target mainbody output by the **mainbody detection** module, and input it into the feature extraction model to obtain the corresponding feature vector, which is used as the representation feature of the image for subsequent retrieval.
- Vector retrieval: Receive one or more feature vectors output by the **feature extraction** module, and retrieve them one by one in the vector library, finally return the retrieval result. This module does not require additional training and can be used by installing the third-party open source faiss retrieval library

In the recognition system, one of the most important modules is the feature extraction model. The generalization of feature extraction model directly affects the quality of the vectors in the retrieval library and the vectors to be retrieved. Therefore, we will introduce feature extraction model below in 5 parts.

- Backbone
    The Backbone uses PP_LCNet_x2_5, which explores many effective structural design schemes for the performance optimization of Intel CPU. Finally, the backbone further improves the performance of the model without increasing the reasoning time, and finally greatly exceeds the existing SOTA model.

- Neck

    The Neck part uses FC Layer to reduce the dimension of features extracted by the Backbone, thus reducing the cost and computation of feature storage.

- Head

    The Head part uses ArcMargin. During training, margin was specified to increase the Angle difference between similar features before classification, so as to further improve the characterization ability of feature extraction.

- Loss

    The Loss adopts Cross entropy loss, using classification loss to optimize the network during training.See the General Identification Profile for detailed configuration files.

## 5. Note
PP-ShiTu is looking for the most cost-effective image recognition solution in industrial practice. However, considering that the datasets of different recognition scenarios have their own distribution characteristics, as well as the limitations of software and hardware during training, it is difficult to integrate all datasets at one time. Therefore, it is recommended that users, after understanding the characteristics of your actual datasets, fine-tune or even make an further development on your own datasets based on the PP-ShiTu pre-training model and training configuration, in order to obtain better performance and generalization.

## 6. Reference
```log
@article{cui2021pp,
    title={PP-LCNet: A Lightweight CPU Convolutional Neural Network},
    author={Cui, Cheng and Gao, Tingquan and Wei, Shengyu and Du, Yuning and Guo, Ruoyu and Dong, Shuilong and Lu, Bin and Zhou, Ying and Lv, Xueying and Liu, Qiwen and others},
    journal={arXiv preprint arXiv:2109.15099},
    year={2021}
}

@article{wei2021pp,
    title={PP-ShiTu: A Practical Lightweight Image Recognition System},
    author={Wei, Shengyu and Guo, Ruoyu and Cui, Cheng and Lu, Bin and Dong, Shuilong and Gao, Tingquan and Du, Yuning and Zhou, Ying and Lyu, Xueying and Liu, Qiwen and others},
    journal={arXiv preprint arXiv:2111.00775},
    year={2021}
}
```

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 